In [1]:
import pandas as pd, ipywidgets as widgets, numpy as np, matplotlib.pyplot as plt, io
from adjustText import adjust_text
raw_uploader, eln_uploader = widgets.FileUpload(multiple = False), widgets.FileUpload(multiple = False)
display("Upload iQue3 Data (csv)")
display(raw_uploader)
display("Upload ELN w/ Antibodies Sheet (xlsx)")
display(eln_uploader)

'Upload iQue3 Data (csv)'

FileUpload(value={}, description='Upload')

'Upload ELN w/ Antibodies Sheet (xlsx)'

FileUpload(value={}, description='Upload')

In [3]:
iquename, elnname = list(raw_uploader.value.keys())[0], list(eln_uploader.value.keys())[0]
iquename
iquedata = pd.read_csv(io.BytesIO(raw_uploader.value[iquename]['content']))
eln = pd.read_excel(io.BytesIO(eln_uploader.value[elnname]['content']))
ab_sheet = pd.read_excel(io.BytesIO(eln_uploader.value[elnname]['content']), sheet_name = "Antibody Plate Map")
iquedata

,Experiment,Analysis,Plate,Export Date and Time,Well ID,Row,Column,Well Type,Count of All Wells,Median BL1-H of singlets,Median RL1-H of singlets,Count of singlets
0,22_04_26 GLP1R TAO Primary Screen Binding,Analysis 1,Plate 1 redo,4/26/2022 4:11:25 PM,A01,A,1,Positive,6973,22955.0,213134.0,326
1,22_04_26 GLP1R TAO Primary Screen Binding,Analysis 1,Plate 1 redo,4/26/2022 4:11:25 PM,B01,B,1,Positive,4087,25404.0,217930.0,223
2,22_04_26 GLP1R TAO Primary Screen Binding,Analysis 1,Plate 1 redo,4/26/2022 4:11:25 PM,C01,C,1,Positive,3860,24793.0,181442.0,195
3,22_04_26 GLP1R TAO Primary Screen Binding,Analysis 1,Plate 1 redo,4/26/2022 4:11:25 PM,D01,D,1,Positive,3567,27937.0,167344.0,151
4,22_04_26 GLP1R TAO Primary Screen Binding,Analysis 1,Plate 1 redo,4/26/2022 4:11:25 PM,E01,E,1,Positive,3625,22873.0,121031.5,218
...,...,...,...,...,...,...,...,...,...,...,...,...
379,22_04_26 GLP1R TAO Primary Screen Binding,Analysis 1,Plate 1 redo,4/26/2022 4:11:25 PM,L24,L,24,Negative,1268,13856.0,67168.5,34
380,22_04_26 GLP1R TAO Primary Screen Binding,Analysis 1,Plate 1 redo,4/26/2022 4:11:25 PM,M24,M,24,Negative,892,15620.0,25527.0,25
381,22_04_26 GLP1R TAO Primary Screen Binding,Analysis 1,Plate 1 redo,4/26/2022 4:11:25 PM,N24,N,24,Negative,1577,13500.5,19546.0,274
382,22_04_26 GLP1R TAO Primary Screen Binding,Analysis 1,Plate 1 redo,4/26/2022 4:11:25 PM,O24,O,24,Negative,2051,13361.0,179164.5,224


In [4]:
data_cleaned = iquedata[["Plate", "Well ID", "Well Type", "Median RL1-H of singlets"]]
data_cleaned

,Plate,Well ID,Well Type,Median RL1-H of singlets
0,Plate 1 redo,A01,Positive,213134.0
1,Plate 1 redo,B01,Positive,217930.0
2,Plate 1 redo,C01,Positive,181442.0
3,Plate 1 redo,D01,Positive,167344.0
4,Plate 1 redo,E01,Positive,121031.5
...,...,...,...,...
379,Plate 1 redo,L24,Negative,67168.5
380,Plate 1 redo,M24,Negative,25527.0
381,Plate 1 redo,N24,Negative,19546.0
382,Plate 1 redo,O24,Negative,179164.5


In [48]:
#Return data frame with only samples of the well type specified
def clean_for_well_type(well_type_string):
    return data_cleaned[data_cleaned["Well Type"] == well_type_string]

#Average function that takes a string indicating the well type of interest (marked in iQue3 software, or manually changed
#Separates n=1 and n=2 replicates from 384 well format, resets indeces, and averages
#Returns a series with the averaged MFI ratios, removing the replicate wells
def average_MFI(well_type_string):
    selected_wells = clean_for_well_type(well_type_string)
    MFIs = selected_wells["Median RL1-H of singlets"]
    firsts = MFIs.iloc[::2]
    seconds = MFIs.iloc[1::2]
    firsts.reset_index(inplace = True, drop = True)
    seconds.reset_index(inplace = True, drop = True)
    average = firsts + seconds
    average = average/2
    return average

#Calculates the MFI ratio given a positive and negative series, 
#and appends the MFI ratio to the original data frame, excluding the negative population
def calculate_MFI_Ratio(positive, negative):
    MFI_ratio = positive/negative
    original_positive = clean_for_well_type("Positive")
    original_positive = original_positive[::2]
    original_positive.reset_index(inplace = True, drop = True)
    original_positive["MFI Ratio"] = MFI_ratio
    MFIs_Antibody = pd.concat([original_positive, ab_sheet], axis = 1)
    MFIs_Antibody.index += 1
    return MFIs_Antibody[["Plate", "Well ID", "Well Type", "MFI Ratio", "DNA", "Protein Concentration"]]
    

In [49]:
averagePositive = average_MFI("Positive")
averageNegative = average_MFI("Negative")

In [50]:
#Create new DF for MFI ratios(don't modify originals)
MFI_Ratios_Antibody = calculate_MFI_Ratio(averagePositive, averageNegative)
MFI_Ratios_Antibody
MFI_Ratios_Antibody.to_excel("Flow Binding MFI Ratios.xlsx")
AllLeadsSorted = MFI_Ratios_Antibody.sort_values(by="MFI Ratio", ascending = False)
AllLeadsSorted

,Plate,Well ID,Well Type,MFI Ratio,DNA,Protein Concentration
1,Plate 1 redo,A01,Positive,1.425391,TB222-001,0.90
2,Plate 1 redo,C01,Positive,1.888699,TB222-002,0.31
3,Plate 1 redo,E01,Positive,1.655639,TB222-003,0.49
4,Plate 1 redo,G01,Positive,1.746913,TB222-004,0.42
5,Plate 1 redo,I01,Positive,1.863327,TB222-005,0.46
...,...,...,...,...,...,...
92,Plate 1 redo,G12,Positive,1.039097,TB222-092,0.12
93,Plate 1 redo,I12,Positive,4.163307,TB222-093,0.44
94,Plate 1 redo,K12,Positive,1.964486,TB222-094,0.03
95,Plate 1 redo,M12,Positive,2.115413,R&D MAB,0.50


In [ ]:
# #Sort DF by highest MFI ratio (top hits)
# AllLeadsSorted = MFIRatio_Antibody_cleaned.sort_values(by="MFI Ratio", ascending = False)
# Top10 = MFIRatio_Antibody_cleaned.nlargest(10, "MFI Ratio")
# #Top10 = Top10.append(MFIRatio_Antibody_cleaned.loc[MFIRatio_Antibody_cleaned["DNA"] == "SC-175-017"])
# fig, ax = plt.subplots()
# ax.scatter(x = AllLeadsSorted["Protein Concentration (ug/uL)"], y=AllLeadsSorted["MFI Ratio"], alpha = .2, color = "blue")
# ax.scatter(x = Top10["Protein Concentration (ug/uL)"], y = Top10["MFI Ratio"], alpha = .8, color = "blue", label = "Top Hits")
# ax.set_title("CD40 Primary Screen")
# ax.set_xlabel("Concentration (mg/ml)")
# ax.set_ylabel("MFI Ratio \n Transfected/Untransfected")
# ax.legend()

# texts = []
# for row in Top10.itertuples(index = False):
#     ab, conc, MFI = row[4], row[5], row[3]
#     ax.annotate(row[4], xy = (conc, MFI), textcoords = "offset points", xytext = (8, -2), ha ='left')
#     #texts.append(plt.text(ab, conc, MFI))